In [15]:
import os
import sys
import gzip
import code
import urllib
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow.python.platform
import matplotlib.image as mpimg
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [16]:
%reload_ext autoreload
%autoreload 2

In [17]:
sys.path.append('./')
from helpers.helpers import *
from helpers.mask_to_submission import *

In [18]:
NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 80
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16  # 64
NUM_EPOCHS = 10
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

In [54]:
data_dir = 'data/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract patches into numpy arrays. Dim: (nb patch, 16, 16, 3)
training_data = extract_data(train_data_filename, 100)
training_labels = extract_labels(train_labels_filename, 100)

Loaded 100 training images
Loaded 100 groudtruth images


In [55]:
X_train, X_validation, y_train, y_validation = train_test_split(training_data, training_labels,\
                                                    train_size= TRAINING_SIZE/100, random_state=SEED)

print(X_train.shape)
print(X_validation.shape)

(50000, 16, 16, 3)
(12500, 16, 16, 3)


In [47]:
num_epochs = NUM_EPOCHS

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(X_train.shape)
X_train = X_train[new_indices, :, :, :]
y_train = y_train[new_indices]

train_size = y_train.shape[0]

c0 = 0
c1 = 0
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [48]:
X_train.shape

(50000, 16, 16, 3)

In [23]:
if RESTORE_MODEL:
    # It can be used to reconstruct the model identically.
    model = models.load_model("U-net_save/")

In [102]:
# if not RESTORE_MODEL:
#     def down_block(model, filters, kernel_size=(3, 3), padding="same", strides=1):
#         model.add(layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu"))
#         model.add(layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu"))
#         model.add(layers.MaxPool2D((2, 2), (2, 2)))

#     def up_block(model, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
#         model.add(layers.UpSampling2D((2, 2)))
#         model.add(layers.Concatenate()([us, skip]))
#         model.add(layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu"))
#         model.add(layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu"))

#     def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
#         model.add(layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu"))
#         model.add(layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu"))

#     f = [16, 32, 64, 128, 256]
#     model = models.Sequential(
#         down_block()
#     )
    

In [49]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = layers.UpSampling2D((2, 2))(x)
    concat = layers.Concatenate()([us, skip])
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [56]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = layers.Input((IMG_PATCH_SIZE, IMG_PATCH_SIZE, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    a0 = layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    a1 = layers.Flatten()(a0)
    a2 = layers.Dense(64, activation='relu')(a1)
    outputs = layers.Dense(2)(a2)
    model = models.Model(inputs, outputs)
    return model

In [57]:
model = UNet()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 16, 16, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 16, 16, 16)   448         input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_134 (Conv2D)             (None, 16, 16, 16)   2320        conv2d_133[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_28 (MaxPooling2D) (None, 8, 8, 16)     0           conv2d_134[0][0]                 
____________________________________________________________________________________________

In [58]:
if not RESTORE_MODEL: 
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs = NUM_EPOCHS ,validation_data=(X_validation, y_validation))

Epoch 1/10
1563/1563 [==============================] - 231s 148ms/step - loss: 0.5439 - accuracy: 0.7405 - val_loss: 0.5204 - val_accuracy: 0.7405
Epoch 2/10
1563/1563 [==============================] - 234s 150ms/step - loss: 0.5040 - accuracy: 0.7447 - val_loss: 0.4754 - val_accuracy: 0.7600
Epoch 3/10
1563/1563 [==============================] - 213s 136ms/step - loss: 0.4326 - accuracy: 0.7876 - val_loss: 0.4190 - val_accuracy: 0.7969
Epoch 4/10
1563/1563 [==============================] - 210s 135ms/step - loss: 0.4041 - accuracy: 0.8042 - val_loss: 0.3978 - val_accuracy: 0.8074
Epoch 5/10
1563/1563 [==============================] - 217s 139ms/step - loss: 0.3884 - accuracy: 0.8136 - val_loss: 0.4004 - val_accuracy: 0.8038
Epoch 6/10
1563/1563 [==============================] - 229s 147ms/step - loss: 0.3806 - accuracy: 0.8190 - val_loss: 0.3853 - val_accuracy: 0.8174
Epoch 7/10
1563/1563 [==============================] - 258s 165ms/step - loss: 0.3704 - accuracy: 0.8244 - val_

In [145]:
if not RESTORE_MODEL:
    model.save("U-net_save/")

INFO:tensorflow:Assets written to: handmade_cnn_save/assets


In [132]:
pred_train = model.predict(X_train)
print("Training error rate: {:.2f}%".format(error_rate(pred_train, y_train)))

Training error rate: 9.66%


In [125]:
pred_validation = model.predict(X_validation)
print("Validation error rate: {:.2f}%".format(error_rate(pred_validation, y_validation)))

Validation error rate: 17.78%


In [144]:
F1_score(y_train, pred_train)

0.7976718867766519

In [146]:
print("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE + 1):
    pimg = get_prediction_with_groundtruth(model, train_data_filename, i)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(model, train_data_filename, i)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")  

Running prediction on training set


In [179]:
predict_test_masks(model)

Running prediction on test set


In [186]:
masks_to_submission("submission.csv", "predictions_test/")

predictions_test/test_14.png
predictions_test/test_28.png
predictions_test/test_29.png
predictions_test/test_15.png
predictions_test/test_9.png
predictions_test/test_17.png
predictions_test/test_16.png
predictions_test/test_8.png
predictions_test/test_12.png
predictions_test/test_13.png
predictions_test/test_39.png
predictions_test/test_11.png
predictions_test/test_10.png
predictions_test/test_38.png
predictions_test/test_48.png
predictions_test/test_49.png
predictions_test/test_42.png
predictions_test/test_43.png
predictions_test/test_41.png
predictions_test/test_40.png
predictions_test/test_44.png
predictions_test/test_50.png
predictions_test/test_45.png
predictions_test/test_47.png
predictions_test/test_46.png
predictions_test/test_3.png
predictions_test/test_35.png
predictions_test/test_21.png
predictions_test/test_20.png
predictions_test/test_34.png
predictions_test/test_2.png
predictions_test/test_22.png
predictions_test/test_36.png
predictions_test/test_37.png
predictions_test/t